# 제품 이상여부 판별 프로젝트

## 1. 데이터 불러오기

### 필수 라이브러리

In [18]:
import os
from pprint import pprint
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

### 데이터 읽어오기

### 엑셀 파일들 읽어오기

In [5]:
ROOT_DIR = r"C:\Users\RYU\Desktop\대기업 가보자!!!!!!!!!!!!!!!!!!!!!!!!!\lg aimers python 5기"
RANDOM_STATE = 110

# Load data
train_data = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
test_data = pd.read_csv(os.path.join(ROOT_DIR, 'test.csv'))
train_data

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,CURE END POSITION X Judge Value_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,NaN,NaN,...,127,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,NaN,NaN,...,185,NaN,NaN,1,NaN,NaN,0.0,NaN,NaN,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,NaN,NaN,...,73,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,NaN,NaN,...,268,NaN,NaN,1,NaN,NaN,0.0,NaN,NaN,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,NaN,NaN,...,121,NaN,NaN,1,NaN,NaN,0.0,NaN,NaN,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,NaN,NaN,...,318,NaN,NaN,1,NaN,NaN,0.0,NaN,NaN,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,NaN,NaN,...,197,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,NaN,NaN,...,27,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,NaN,NaN,...,117,NaN,NaN,1,NaN,NaN,0.0,NaN,NaN,Normal


### 데이터 

In [8]:
# Drop columns with more than half of the values missing
drop_cols = []
for column in train_data.columns:
    if (train_data[column].notnull().sum() // 2) < train_data[
        column
    ].isnull().sum():
        drop_cols.append(column)
train_data = train_data.drop(drop_cols, axis=1)


train_data

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,...,Head Clean Position Y Collect Result_Fill2,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,2.5,-90,...,50,91.8,270.0,50,114.612,19.9,7.0,127,1,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,2.5,-90,...,50,91.8,270.0,50,85.000,19.6,7.0,185,1,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,12.5,90,...,50,91.8,270.0,50,114.612,19.8,10.0,73,1,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,12.5,90,...,50,91.8,270.0,50,85.000,19.9,12.0,268,1,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,2.5,-90,...,50,91.8,270.0,50,85.000,19.7,8.0,121,1,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,2.5,-90,...,50,91.8,270.0,50,85.000,19.2,1.0,318,1,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,12.5,90,...,50,91.8,270.0,50,114.612,20.5,14.0,197,1,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,2.5,-90,...,50,91.8,270.0,50,85.000,19.7,1.0,27,1,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,12.5,90,...,50,91.8,270.0,50,85.000,20.1,13.0,117,1,Normal


In [13]:
# 삭제할 열 목록
columns_to_drop = ['Wip Line_Dam', 'Process Desc._Dam', 'Model.Suffix_Dam','Workorder_Dam','Insp. Seq No._Dam','Insp Judge Code_Dam',
                   'CURE END POSITION X Collect Result_Dam','CURE END POSITION Z Collect Result_Dam','CURE END POSITION Θ Collect Result_Dam',
                   'CURE STANDBY POSITION X Collect Result_Dam','CURE STANDBY POSITION Z Collect Result_Dam','CURE STANDBY POSITION Θ Collect Result_Dam',
                   'CURE START POSITION X Collect Result_Dam','CURE START POSITION Z Collect Result_Dam','CURE START POSITION Θ Collect Result_Dam',
                   'HEAD Standby Position X Collect Result_Dam','HEAD Standby Position Y Collect Result_Dam','HEAD Standby Position Z Collect Result_Dam'
                   
                   'Wip Line_AutoClave','Process Desc._AutoClave','Equipment_AutoClave','Model.Suffix_AutoClave','Workorder_AutoClave',
                   'Insp. Seq No._AutoClave','Insp Judge Code_AutoClave','1st Pressure Judge Value_AutoClave','2nd Pressure Judge Value_AutoClave',
                   '3rd Pressure Judge Value_AutoClave',
                   
                   'Wip Line_Fill1','Process Desc._Fill1','Equipment_Fill1','Model.Suffix_Fill1','Workorder_Fill1','Insp. Seq No._Fill1',
                   'Insp Judge Code_Fill1','HEAD Standby Position X Collect Result_Fill1','HEAD Standby Position Y Collect Result_Fill1',
                   'HEAD Standby Position Z Collect Result_Fill1','Head Clean Position X Collect Result_Fill1','Head Clean Position Y Collect Result_Fill1',
                   'Head Clean Position Z Collect Result_Fill1'
                   
                   'Wip Line_Fill2','Process Desc._Fill2','Equipment_Fill2','Model.Suffix_Fill2','Workorder_Fill2','Insp. Seq No._Fill2',
                   'Insp Judge Code_Fill2','DISCHARGED SPEED OF RESIN Collect Result_Fill2'
                   'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill2','DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill2',
                   'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill2','Dispense Volume(Stage1) Collect Result_Fill2'
                   'Dispense Volume(Stage2) Collect Result_Fill2','Dispense Volume(Stage3) Collect Result_Fill2',
                   'HEAD Standby Position X Collect Result_Fill2','HEAD Standby Position Y Collect Result_Fill2',
                   'HEAD Standby Position Z Collect Result_Fill2','Head Clean Position X Collect Result_Fill2',
                   'Head Clean Position Y Collect Result_Fill2','Head Clean Position Z Collect Result_Fill2']

# 열 삭제
train_data = train_data.drop(columns=columns_to_drop, errors='ignore')

In [14]:
train_data

,Equipment_Dam,CURE SPEED Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,Dispense Volume(Stage1) Collect Result_Dam,Dispense Volume(Stage2) Collect Result_Dam,Dispense Volume(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam,...,HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2,HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,target
0,Dam dispenser #1,100,16,14.9,8.4,14.7,1.04,0.58,1.02,550.5,...,243.7,243.7,270.0,50,114.612,19.9,7.0,127,1,Normal
1,Dam dispenser #1,70,10,21.3,4.9,21.3,1.49,0.34,1.49,550.3,...,243.7,243.7,270.0,50,85.000,19.6,7.0,185,1,Normal
2,Dam dispenser #2,85,16,14.7,8.5,14.7,1.61,0.93,1.61,162.4,...,243.5,243.5,270.0,50,114.612,19.8,10.0,73,1,Normal
3,Dam dispenser #2,70,10,21.3,8.4,21.3,1.49,0.58,1.49,162.4,...,243.5,243.5,270.0,50,85.000,19.9,12.0,268,1,Normal
4,Dam dispenser #1,70,10,9.7,4.9,9.6,0.67,0.34,0.67,549.0,...,243.7,243.7,270.0,50,85.000,19.7,8.0,121,1,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,Dam dispenser #1,70,10,17.0,5.0,17.0,1.19,0.35,1.19,550.3,...,243.7,243.7,270.0,50,85.000,19.2,1.0,318,1,Normal
40502,Dam dispenser #2,100,16,14.9,8.5,14.7,1.04,0.59,1.02,163.5,...,243.5,243.5,270.0,50,114.612,20.5,14.0,197,1,Normal
40503,Dam dispenser #1,100,16,14.2,8.2,14.3,0.99,0.57,1.00,550.6,...,243.7,243.7,270.0,50,85.000,19.7,1.0,27,1,Normal
40504,Dam dispenser #2,70,10,9.7,4.9,9.7,0.67,0.34,0.67,162.4,...,243.5,243.5,270.0,50,85.000,20.1,13.0,117,1,Normal


In [17]:
# 'object' 타입 열 찾기
object_columns = train_data.select_dtypes(include=['object']).columns
print("Object columns:", object_columns)

Object columns: Index(['target'], dtype='object')


In [16]:
#Equipment
train_data['Equipment_Dam'] = train_data['Equipment_Dam'].apply(lambda x: 1 if x == 'Dam dispenser #1'  else 0)

#Chamber Temp. Judge Value_AutoClave
train_data['Chamber Temp. Judge Value_AutoClave'] = train_data['Chamber Temp. Judge Value_AutoClave'].apply(lambda x: 1 if x == 'OK'  else 0)

#target
train_data['target'] = train_data['target'].apply(lambda x : 1 if x =='AbNormal' else 0 )

In [44]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

# 데이터 로딩 및 전처리
train_data = pd.read_csv('train_data.csv')
X = train_data.drop(columns=['target'])
y = train_data['target']

# 데이터 정규화
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# 시퀀스 생성
def create_sequences(data, targets, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        label = targets[i + seq_length]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

seq_length = 10
X_sequences, y_sequences = create_sequences(X_scaled, y, seq_length)

# 훈련 데이터와 검증 데이터로 분할
X_train, X_val, y_train, y_val = train_test_split(X_sequences, y_sequences, test_size=0.2, shuffle=False)

# 클래스 가중치 정의
class_weight = {0: 1., 1: 10.}

# GRU 모델 정의
model = Sequential([
    GRU(64, activation='tanh', input_shape=(seq_length, X_sequences.shape[2])),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 훈련
history = model.fit(X_train, y_train, class_weight=class_weight, epochs=50, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# 모델 평가 및 예측
y_pred = model.predict(X_val)
y_pred_binary = (y_pred > 0.5).astype(int)

# 성능 평가
print(classification_report(y_val, y_pred_binary))


ModuleNotFoundError: No module named 'distutils'

In [49]:
pip install --upgrade setuptools

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [51]:
pip uninstall setuptools
